In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import os
from sklearn.metrics import roc_curve
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Embedding, Activation, LSTM, SimpleRNN, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from tensorflow.keras import backend as K
import tensorflow as tf
import tensorflow_hub as hub
print("TensorFlow Version:",tf.__version__)
print("Hub version: ",hub.__version__)
# Params for bert model and tokenization


TensorFlow Version: 2.1.0
Hub version:  0.8.0


In [3]:
class LoadingData():
            
    def __init__(self):
        train_file_path = os.path.join("..","input","nlp-benchmarking-data-for-intent-and-entity","benchmarking_data","Train")
        validation_file_path = os.path.join("..","input","nlp-benchmarking-data-for-intent-and-entity","benchmarking_data","Validate")
        category_id = 0
        self.cat_to_intent = {}
        self.intent_to_cat = {}
        
        for dirname, _, filenames in os.walk(train_file_path):
            for filename in filenames:
                file_path = os.path.join(dirname, filename)
                intent_id = filename.replace(".json","")
                self.cat_to_intent[category_id] = intent_id
                self.intent_to_cat[intent_id] = category_id
                category_id+=1
        print(self.cat_to_intent)
        print(self.intent_to_cat)
        '''Training data'''
        training_data = list() 
        for dirname, _, filenames in os.walk(train_file_path):
            for filename in filenames:
                file_path = os.path.join(dirname, filename)
                intent_id = filename.replace(".json","")
                training_data+=self.make_data_for_intent_from_json(file_path,intent_id,self.intent_to_cat[intent_id])
        self.train_data_frame = pd.DataFrame(training_data, columns =['query', 'intent','category'])   
        
        self.train_data_frame = self.train_data_frame.sample(frac = 1)


        
        '''Validation data'''
        validation_data = list()    
        for dirname, _, filenames in os.walk(validation_file_path):
            for filename in filenames:
                file_path = os.path.join(dirname, filename)
                intent_id = filename.replace(".json","")
                validation_data +=self.make_data_for_intent_from_json(file_path,intent_id,self.intent_to_cat[intent_id])                
        self.validation_data_frame = pd.DataFrame(validation_data, columns =['query', 'intent','category'])

        self.validation_data_frame = self.validation_data_frame.sample(frac = 1)
        
        
    def make_data_for_intent_from_json(self,json_file,intent_id,cat):
        json_d = json.load(open(json_file))         
        
        json_dict = json_d[intent_id]

        sent_list = list()
        for i in json_dict:
            each_list = i['data']
            sent =""
            for i in each_list:
                sent = sent + i['text']+ " "
            sent =sent[:-1]
            for i in range(3):
                sent = sent.replace("  "," ")
            sent_list.append((sent,intent_id,cat))
        return sent_list
            

In [4]:
load_data_obj = LoadingData()

{0: 'BookRestaurant', 1: 'SearchScreeningEvent', 2: 'RateBook', 3: 'GetWeather', 4: 'AddToPlaylist', 5: 'PlayMusic', 6: 'SearchCreativeWork'}
{'BookRestaurant': 0, 'SearchScreeningEvent': 1, 'RateBook': 2, 'GetWeather': 3, 'AddToPlaylist': 4, 'PlayMusic': 5, 'SearchCreativeWork': 6}


In [5]:
load_data_obj.train_data_frame.head()

,query,intent,category
12271,find the painting Sleeping in Your Hand,SearchCreativeWork,6
11003,Open Deezer and play Inyeccion Musical,PlayMusic,5
3109,Find the schedule for The Cup Winner at the cl...,SearchScreeningEvent,1
13484,"Can you find me the game , Super Scription of ...",SearchCreativeWork,6
2948,will Dick Tracy e il gas misterioso start twen...,SearchScreeningEvent,1


In [6]:
load_data_obj.train_data_frame

,query,intent,category
12271,find the painting Sleeping in Your Hand,SearchCreativeWork,6
11003,Open Deezer and play Inyeccion Musical,PlayMusic,5
3109,Find the schedule for The Cup Winner at the cl...,SearchScreeningEvent,1
13484,"Can you find me the game , Super Scription of ...",SearchCreativeWork,6
2948,will Dick Tracy e il gas misterioso start twen...,SearchScreeningEvent,1
...,...,...,...
1418,book a spot at a restaurant within walking dis...,BookRestaurant,0
13379,Find me the Martin Morning saga,SearchCreativeWork,6
4833,Rate A Handful of Darkness a value of 3,RateBook,2
6101,weather forcast for current location now,GetWeather,3


In [7]:
load_data_obj.validation_data_frame.head().values

array([['Play something from 2004 by Imogen Heap on spotify',
        'PlayMusic', 5],
       ['what are the times for The Gingerbread Man',
        'SearchScreeningEvent', 1],
       ['I want to book Tupelo Honey Cafe in New Jersey for five people.',
        'BookRestaurant', 0],
       ['Add jack white to my playlist This Is Shakira', 'AddToPlaylist',
        4],
       ['Rate the chronicle current 1 star', 'RateBook', 2]], dtype=object)

In [8]:
load_data_obj.train_data_frame.head()

,query,intent,category
12271,find the painting Sleeping in Your Hand,SearchCreativeWork,6
11003,Open Deezer and play Inyeccion Musical,PlayMusic,5
3109,Find the schedule for The Cup Winner at the cl...,SearchScreeningEvent,1
13484,"Can you find me the game , Super Scription of ...",SearchCreativeWork,6
2948,will Dick Tracy e il gas misterioso start twen...,SearchScreeningEvent,1


## LSTM

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Initialize the tokenizer
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(load_data_obj.train_data_frame['query'])

# Convert text to sequence of integers
train_sequences = tokenizer.texts_to_sequences(load_data_obj.train_data_frame['query'])
validation_sequences = tokenizer.texts_to_sequences(load_data_obj.validation_data_frame['query'])

# Pad sequences to ensure uniform length
max_length = max([len(x) for x in train_sequences])
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post')
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding='post')


In [10]:
import pandas as pd
from tensorflow.keras.utils import to_categorical

# Convert labels to one-hot encoding
train_labels = to_categorical(load_data_obj.train_data_frame['category'])
validation_labels = to_categorical(load_data_obj.validation_data_frame['category'])


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

# Assuming max_length is defined here, e.g., max_length = max([len(x) for x in train_sequences])
max_length = max([len(x) for x in train_sequences])

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=32, input_length=max_length))  # Adjusted output_dim
model.add(Bidirectional(LSTM(128, return_sequences=False)))  # Increased LSTM units and added Bidirectional layer
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))  # Adjusted dropout rate
model.add(Dense(len(train_labels[0]), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 35, 32)            160000    
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               164864    
_________________________________________________________________
dense (Dense)                (None, 64)                16448     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 455       
Total params: 341,767
Trainable params: 341,767
Non-trainable params: 0
_________________________________________________________________


In [12]:
history = model.fit(train_padded, train_labels, epochs=10, validation_data=(validation_padded, validation_labels))


Train on 13784 samples, validate on 700 samples
Epoch 1/10
13784/13784 [==============================] - 9s 664us/sample - loss: 0.5113 - accuracy: 0.8249 - val_loss: 0.0794 - val_accuracy: 0.9743
Epoch 2/10
13784/13784 [==============================] - 4s 259us/sample - loss: 0.0565 - accuracy: 0.9836 - val_loss: 0.0587 - val_accuracy: 0.9843
Epoch 3/10
13784/13784 [==============================] - 4s 257us/sample - loss: 0.0336 - accuracy: 0.9912 - val_loss: 0.0650 - val_accuracy: 0.9786
Epoch 4/10
13784/13784 [==============================] - 4s 257us/sample - loss: 0.0246 - accuracy: 0.9941 - val_loss: 0.0675 - val_accuracy: 0.9786
Epoch 5/10
13784/13784 [==============================] - 4s 259us/sample - loss: 0.0164 - accuracy: 0.9954 - val_loss: 0.0683 - val_accuracy: 0.9786
Epoch 6/10
13784/13784 [==============================] - 4s 259us/sample - loss: 0.0104 - accuracy: 0.9972 - val_loss: 0.0961 - val_accuracy: 0.9771
Epoch 7/10
13784/13784 [============================

In [13]:
loss, accuracy = model.evaluate(validation_padded, validation_labels)
print(f'Validation loss: {loss}, Validation accuracy: {accuracy}')


700/700 [==============================] - 0s 114us/sample - loss: 0.0959 - accuracy: 0.9814
Validation loss: 0.09589042356121354, Validation accuracy: 0.9814285635948181


In [14]:

# Create a DataFrame for results
results_df = pd.DataFrame(columns=['model_name', 'validation_accuracy'])

# Insert data into the DataFrame
results_df = results_df.append({'model_name': 'LSTM', 'validation_accuracy': accuracy}, ignore_index=True)

# Display the DataFrame
print(results_df)


  model_name  validation_accuracy
0       LSTM             0.981429


## RANDOM FOREST


In [15]:
train_data_frame=load_data_obj.train_data_frame
validation_data_frame=load_data_obj.validation_data_frame

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
train_tfidf = tfidf_vectorizer.fit_transform(train_data_frame['query'])
validation_tfidf = tfidf_vectorizer.transform(validation_data_frame['query'])

# Encode labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_data_frame['category'])
validation_labels_encoded = label_encoder.transform(validation_data_frame['category'])


In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_classifier.fit(train_tfidf, train_labels_encoded)


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [17]:
# Predict on validation set
validation_predictions = rf_classifier.predict(validation_tfidf)

# Calculate accuracy
validation_accuracy = accuracy_score(validation_labels_encoded, validation_predictions)
print(f'Validation Accuracy of Random Forest: {validation_accuracy * 100:.2f}%')


Validation Accuracy of Random Forest: 97.43%


In [18]:
# Insert data into the DataFrame
results_df = results_df.append({'model_name': 'Random Forest', 'validation_accuracy': validation_accuracy}, ignore_index=True)

# Display the DataFrame
print(results_df)

      model_name  validation_accuracy
0           LSTM             0.981429
1  Random Forest             0.974286


## Logistic Regression

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Initialize the Logistic Regression Classifier
log_reg_classifier = LogisticRegression(max_iter=1000)  # Increase max_iter if the model doesn't converge

# Train the model
log_reg_classifier.fit(train_tfidf, train_labels_encoded)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [20]:
# Predict on validation set
validation_predictions = log_reg_classifier.predict(validation_tfidf)

# Calculate accuracy
validation_accuracy = accuracy_score(validation_labels_encoded, validation_predictions)
print(f'Validation Accuracy of Logistic Regression: {validation_accuracy * 100:.2f}%')


Validation Accuracy of Logistic Regression: 98.29%


In [21]:
# Insert data into the DataFrame
results_df = results_df.append({'model_name': 'Logistic Regression', 'validation_accuracy': validation_accuracy}, ignore_index=True)

# Display the DataFrame
print(results_df)


            model_name  validation_accuracy
0                 LSTM             0.981429
1        Random Forest             0.974286
2  Logistic Regression             0.982857


## GRU

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Initialize the model
model = Sequential()

# Add an Embedding layer
model.add(Embedding(input_dim=5000, output_dim=16, input_length=max_length))

# First GRU layer with Dropout regularization
model.add(GRU(units=50, return_sequences=True, activation='tanh'))
model.add(Dropout(0.2))

# Second GRU layer
model.add(GRU(units=50, return_sequences=True, activation='tanh'))
model.add(Dropout(0.2))

# Third GRU layer
model.add(GRU(units=50, return_sequences=True, activation='tanh'))
model.add(Dropout(0.2))

# Fourth GRU layer
model.add(GRU(units=50, activation='tanh'))
model.add(Dropout(0.2))

# Output layer for classification (units = number of classes, softmax activation)
model.add(Dense(units=len(train_labels[0]), activation='softmax'))  # Adjust the units based on the number of classes

# Compile the model for classification
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 35, 16)            80000     
_________________________________________________________________
gru (GRU)                    (None, 35, 50)            10200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 35, 50)            0         
_________________________________________________________________
gru_1 (GRU)                  (None, 35, 50)            15300     
_________________________________________________________________
dropout_2 (Dropout)          (None, 35, 50)            0         
_________________________________________________________________
gru_2 (GRU)                  (None, 35, 50)            15300     
_________________________________________________________________
dropout_3 (Dropout)          (None, 35, 50)           

In [23]:
# Train the model
history = model.fit(train_padded, train_labels, epochs=10, validation_data=(validation_padded, validation_labels))


Train on 13784 samples, validate on 700 samples
Epoch 1/10
13784/13784 [==============================] - 10s 760us/sample - loss: 1.9478 - accuracy: 0.1391 - val_loss: 1.9467 - val_accuracy: 0.1429
Epoch 2/10
13784/13784 [==============================] - 5s 391us/sample - loss: 1.8734 - accuracy: 0.1669 - val_loss: 1.4000 - val_accuracy: 0.4100
Epoch 3/10
13784/13784 [==============================] - 5s 398us/sample - loss: 0.5268 - accuracy: 0.7994 - val_loss: 0.3140 - val_accuracy: 0.9143
Epoch 4/10
13784/13784 [==============================] - 5s 394us/sample - loss: 0.1833 - accuracy: 0.9509 - val_loss: 0.2405 - val_accuracy: 0.9386
Epoch 5/10
13784/13784 [==============================] - 5s 394us/sample - loss: 0.1194 - accuracy: 0.9697 - val_loss: 0.1859 - val_accuracy: 0.9529
Epoch 6/10
13784/13784 [==============================] - 5s 392us/sample - loss: 0.0763 - accuracy: 0.9808 - val_loss: 0.1908 - val_accuracy: 0.9529
Epoch 7/10
13784/13784 [===========================

In [24]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_padded, validation_labels)
print(f'Validation loss: {loss}, Validation accuracy: {accuracy}')


700/700 [==============================] - 0s 151us/sample - loss: 0.1290 - accuracy: 0.9700
Validation loss: 0.12895991353052003, Validation accuracy: 0.9700000286102295


In [25]:
# Insert data into the DataFrame
results_df = results_df.append({'model_name': 'GRU', 'validation_accuracy': accuracy}, ignore_index=True)

# Display the DataFrame
print(results_df)


            model_name  validation_accuracy
0                 LSTM             0.981429
1        Random Forest             0.974286
2  Logistic Regression             0.982857
3                  GRU             0.970000


## RNN 

In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Initialize the model
model = Sequential()

# Add an Embedding layer
model.add(Embedding(input_dim=5000, output_dim=16, input_length=max_length))

# Add a SimpleRNN layer
model.add(SimpleRNN(units=64, return_sequences=True))
model.add(Dropout(0.2))

# Add another SimpleRNN layer
model.add(SimpleRNN(units=64))
model.add(Dropout(0.2))

# Add the output Dense layer with softmax activation for multi-class classification
model.add(Dense(units=len(train_labels[0]), activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 35, 16)            80000     
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 35, 64)            5184      
_________________________________________________________________
dropout_5 (Dropout)          (None, 35, 64)            0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 64)                8256      
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 7)                 455       
Total params: 93,895
Trainable params: 93,895
Non-trainable params: 0
__________________________________________________

In [27]:
# Train the model
history = model.fit(train_padded, train_labels, epochs=10, validation_data=(validation_padded, validation_labels))


Train on 13784 samples, validate on 700 samples
Epoch 1/10
13784/13784 [==============================] - 16s 1ms/sample - loss: 0.4116 - accuracy: 0.8693 - val_loss: 0.1726 - val_accuracy: 0.9514
Epoch 2/10
13784/13784 [==============================] - 14s 1ms/sample - loss: 0.1275 - accuracy: 0.9642 - val_loss: 0.1749 - val_accuracy: 0.9471
Epoch 3/10
13784/13784 [==============================] - 14s 992us/sample - loss: 0.0850 - accuracy: 0.9769 - val_loss: 0.2253 - val_accuracy: 0.9400
Epoch 4/10
13784/13784 [==============================] - 14s 997us/sample - loss: 0.0687 - accuracy: 0.9808 - val_loss: 0.2269 - val_accuracy: 0.9486
Epoch 5/10
13784/13784 [==============================] - 14s 991us/sample - loss: 0.0452 - accuracy: 0.9873 - val_loss: 0.1521 - val_accuracy: 0.9614
Epoch 6/10
13784/13784 [==============================] - 14s 990us/sample - loss: 0.0580 - accuracy: 0.9849 - val_loss: 0.1364 - val_accuracy: 0.9700
Epoch 7/10
13784/13784 [==========================

In [28]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_padded, validation_labels)
print(f'Validation loss: {loss}, Validation accuracy: {accuracy}')


700/700 [==============================] - 0s 197us/sample - loss: 0.1652 - accuracy: 0.9586
Validation loss: 0.1652447593957186, Validation accuracy: 0.9585714340209961


In [29]:
# Insert data into the DataFrame
results_df = results_df.append({'model_name': 'RNN', 'validation_accuracy': accuracy}, ignore_index=True)

# Display the DataFrame
print(results_df)


            model_name  validation_accuracy
0                 LSTM             0.981429
1        Random Forest             0.974286
2  Logistic Regression             0.982857
3                  GRU             0.970000
4                  RNN             0.958571


## BERT

In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer


# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')

# Load and preprocess the data
data_train = train_data_frame[['query', 'intent']]
data_train['category'] = pd.Categorical(data_train['intent'])
data_train['intent'] = data_train['category'].cat.codes

# Load and preprocess the data
data_test = validation_data_frame[['query', 'intent']]
data_test['category'] = pd.Categorical(data_test['intent'])
data_test['intent'] = data_test['category'].cat.codes



# Extract the training and testing texts and labels
train_texts = data_train['query'].tolist()
train_labels = data_train['intent'].tolist()
test_texts = data_test['query'].tolist()
test_labels = data_test['intent'].tolist()

max_length = 128  # Adjust based on your dataset or model's max length

train_encodings = tokenizer.batch_encode_plus(
    train_texts, 
    truncation=True, 
    padding='max_length', 
    max_length=max_length, 
    return_tensors="tf",
    pad_to_max_length=True  # Explicitly enforce padding to max_length
)

test_encodings = tokenizer.batch_encode_plus(
    test_texts, 
    truncation=True, 
    padding='max_length', 
    max_length=max_length, 
    return_tensors="tf",
    pad_to_max_length=True  # Explicitly enforce padding to max_length
)



# Convert the labels to one-hot encoding
num_labels = len(data_train['category'].cat.categories)
train_labels_encoded = tf.one_hot(train_labels, num_labels)
test_labels_encoded = tf.one_hot(test_labels, num_labels)

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels_encoded))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), test_labels_encoded))

# Define the model architecture
input_ids = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='attention_mask')
output = model(input_ids, attention_mask=attention_mask)[0]
output = tf.keras.layers.Dense(num_labels, activation='softmax')(output[:, 0, :])  # Pooling the output
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

# Compile and train the model
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']



# Use smaller batch size
batch_size = 20



model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
model.fit(train_dataset.batch(batch_size), epochs=10)

# Evaluate the model
model.evaluate(test_dataset.batch(batch_size))


Train for 690 steps
Epoch 1/10
690/690 [==============================] - 243s 353ms/step - loss: 1.2315 - accuracy: 0.9463
Epoch 2/10
690/690 [==============================] - 228s 330ms/step - loss: 1.1775 - accuracy: 0.9888
Epoch 3/10
690/690 [==============================] - 228s 330ms/step - loss: 1.1755 - accuracy: 0.9904
Epoch 4/10
690/690 [==============================] - 227s 330ms/step - loss: 1.1743 - accuracy: 0.9913
Epoch 5/10
690/690 [==============================] - 227s 330ms/step - loss: 1.1735 - accuracy: 0.9920
Epoch 6/10
690/690 [==============================] - 227s 330ms/step - loss: 1.1715 - accuracy: 0.9941
Epoch 7/10
690/690 [==============================] - 228s 330ms/step - loss: 1.1733 - accuracy: 0.9923
Epoch 8/10
690/690 [==============================] - 228s 330ms/step - loss: 1.1726 - accuracy: 0.9930
Epoch 9/10
690/690 [==============================] - 228s 330ms/step - loss: 1.1720 - accuracy: 0.9935
Epoch 10/10
35/35 [========================

[1.1784067119870867, 0.98714286]

In [31]:
evaluation_results = model.evaluate(test_dataset.batch(batch_size))
accuracy = evaluation_results[1]  # Assuming accuracy is the second metric in the metrics list


35/35 [==============================] - 4s 126ms/step - loss: 1.1784 - accuracy: 0.9871


In [32]:
# Insert data into the DataFrame
results_df = results_df.append({'model_name': 'BERT', 'validation_accuracy': accuracy}, ignore_index=True)

# Display the DataFrame
print(results_df)


            model_name  validation_accuracy
0                 LSTM             0.981429
1        Random Forest             0.974286
2  Logistic Regression             0.982857
3                  GRU             0.970000
4                  RNN             0.958571
5                 BERT             0.987143


## ROBERTA


In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from transformers import TFRobertaModel, RobertaTokenizer

#Load the tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = TFRobertaModel.from_pretrained('roberta-base')



# Load and preprocess the data
data_train = train_data_frame[['query', 'intent']]
data_train['category'] = pd.Categorical(data_train['intent'])
data_train['intent'] = data_train['category'].cat.codes

# Load and preprocess the data
data_test = validation_data_frame[['query', 'intent']]
data_test['category'] = pd.Categorical(data_test['intent'])
data_test['intent'] = data_test['category'].cat.codes



# Extract the training and testing texts and labels
train_texts = data_train['query'].tolist()
train_labels = data_train['intent'].tolist()
test_texts = data_test['query'].tolist()
test_labels = data_test['intent'].tolist()

max_length = 128  # Adjust based on your dataset or model's max length

train_encodings = tokenizer.batch_encode_plus(
    train_texts, 
    truncation=True, 
    padding='max_length', 
    max_length=max_length, 
    return_tensors="tf",
    pad_to_max_length=True  # Explicitly enforce padding to max_length
)

test_encodings = tokenizer.batch_encode_plus(
    test_texts, 
    truncation=True, 
    padding='max_length', 
    max_length=max_length, 
    return_tensors="tf",
    pad_to_max_length=True  # Explicitly enforce padding to max_length
)



# Convert the labels to one-hot encoding
num_labels = len(data_train['category'].cat.categories)
train_labels_encoded = tf.one_hot(train_labels, num_labels)
test_labels_encoded = tf.one_hot(test_labels, num_labels)

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels_encoded))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), test_labels_encoded))

#Define the model architecture
input_ids = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='attention_mask')
output = model(input_ids, attention_mask=attention_mask)[0]
output = tf.keras.layers.Dense(num_labels, activation='softmax')(output[:, 0, :]) # Pooling the output
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

#Compile and train the model
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']


# Use smaller batch size
batch_size = 20



model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
model.fit(train_dataset.batch(batch_size), epochs=10)

# Evaluate the model

evaluation_results = model.evaluate(test_dataset.batch(batch_size))
accuracy = evaluation_results[1]  # Assuming accuracy is the second metric in the metrics list


Train for 690 steps
Epoch 1/10
690/690 [==============================] - 247s 358ms/step - loss: 1.2401 - accuracy: 0.9301
Epoch 2/10
690/690 [==============================] - 230s 334ms/step - loss: 1.1826 - accuracy: 0.9827
Epoch 3/10
690/690 [==============================] - 230s 334ms/step - loss: 1.1777 - accuracy: 0.9880
Epoch 4/10
690/690 [==============================] - 230s 334ms/step - loss: 1.1792 - accuracy: 0.9864
Epoch 5/10
690/690 [==============================] - 230s 334ms/step - loss: 1.1768 - accuracy: 0.9888
Epoch 6/10
690/690 [==============================] - 230s 334ms/step - loss: 1.1757 - accuracy: 0.9897
Epoch 7/10
690/690 [==============================] - 230s 334ms/step - loss: 1.1794 - accuracy: 0.9861
Epoch 8/10
690/690 [==============================] - 230s 334ms/step - loss: 1.1761 - accuracy: 0.9893
Epoch 9/10
690/690 [==============================] - 230s 334ms/step - loss: 1.1753 - accuracy: 0.9901
Epoch 10/10
35/35 [========================

In [34]:
# Insert data into the DataFrame
results_df = results_df.append({'model_name': 'Roberta', 'validation_accuracy': accuracy}, ignore_index=True)

# Display the DataFrame
print(results_df)


            model_name  validation_accuracy
0                 LSTM             0.981429
1        Random Forest             0.974286
2  Logistic Regression             0.982857
3                  GRU             0.970000
4                  RNN             0.958571
5                 BERT             0.987143
6              Roberta             0.990000


## XLnet 

In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from transformers import TFXLNetModel, XLNetTokenizer

# Load the tokenizer and model
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
model = TFXLNetModel.from_pretrained('xlnet-base-cased')




# Load and preprocess the data
data_train = train_data_frame[['query', 'intent']]
data_train['category'] = pd.Categorical(data_train['intent'])
data_train['intent'] = data_train['category'].cat.codes

# Load and preprocess the data
data_test = validation_data_frame[['query', 'intent']]
data_test['category'] = pd.Categorical(data_test['intent'])
data_test['intent'] = data_test['category'].cat.codes



# Extract the training and testing texts and labels
train_texts = data_train['query'].tolist()
train_labels = data_train['intent'].tolist()
test_texts = data_test['query'].tolist()
test_labels = data_test['intent'].tolist()

max_length = 128  # Adjust based on your dataset or model's max length

train_encodings = tokenizer.batch_encode_plus(
    train_texts, 
    truncation=True, 
    padding='max_length', 
    max_length=max_length, 
    return_tensors="tf",
    pad_to_max_length=True  # Explicitly enforce padding to max_length
)

test_encodings = tokenizer.batch_encode_plus(
    test_texts, 
    truncation=True, 
    padding='max_length', 
    max_length=max_length, 
    return_tensors="tf",
    pad_to_max_length=True  # Explicitly enforce padding to max_length
)



# Convert the labels to one-hot encoding
num_labels = len(data_train['category'].cat.categories)
train_labels_encoded = tf.one_hot(train_labels, num_labels)
test_labels_encoded = tf.one_hot(test_labels, num_labels)

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels_encoded))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), test_labels_encoded))

#Define the model architecture
input_ids = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='attention_mask')
output = model(input_ids, attention_mask=attention_mask)[0]
output = tf.keras.layers.Dense(num_labels, activation='softmax')(output[:, 0, :]) # Pooling the output
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

#Compile and train the model
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']


# Use smaller batch size
batch_size = 20



model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
model.fit(train_dataset.batch(batch_size), epochs=10)

# Evaluate the model

evaluation_results = model.evaluate(test_dataset.batch(batch_size))
accuracy = evaluation_results[1]  # Assuming accuracy is the second metric in the metrics list


Train for 690 steps
Epoch 1/10
690/690 [==============================] - 305s 442ms/step - loss: 1.4381 - accuracy: 0.7217
Epoch 2/10
690/690 [==============================] - 291s 422ms/step - loss: 1.1938 - accuracy: 0.9717
Epoch 3/10
690/690 [==============================] - 291s 422ms/step - loss: 1.1863 - accuracy: 0.9794
Epoch 4/10
690/690 [==============================] - 291s 422ms/step - loss: 1.1846 - accuracy: 0.9808
Epoch 5/10
690/690 [==============================] - 291s 422ms/step - loss: 1.1813 - accuracy: 0.9841
Epoch 6/10
690/690 [==============================] - 291s 422ms/step - loss: 1.1816 - accuracy: 0.9835
Epoch 7/10
690/690 [==============================] - 291s 422ms/step - loss: 1.1815 - accuracy: 0.9840
Epoch 8/10
690/690 [==============================] - 291s 422ms/step - loss: 1.1810 - accuracy: 0.9845
Epoch 9/10
690/690 [==============================] - 291s 422ms/step - loss: 1.1816 - accuracy: 0.9836
Epoch 10/10
35/35 [========================

In [36]:
# Insert data into the DataFrame
results_df = results_df.append({'model_name': 'XLnet', 'validation_accuracy': accuracy}, ignore_index=True)

# Display the DataFrame
print(results_df)


            model_name  validation_accuracy
0                 LSTM             0.981429
1        Random Forest             0.974286
2  Logistic Regression             0.982857
3                  GRU             0.970000
4                  RNN             0.958571
5                 BERT             0.987143
6              Roberta             0.990000
7                XLnet             0.982857


## DistilBert


In [37]:
from transformers import TFDistilBertModel, DistilBertTokenizer

#Load the tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')




# Load and preprocess the data
data_train = train_data_frame[['query', 'intent']]
data_train['category'] = pd.Categorical(data_train['intent'])
data_train['intent'] = data_train['category'].cat.codes

# Load and preprocess the data
data_test = validation_data_frame[['query', 'intent']]
data_test['category'] = pd.Categorical(data_test['intent'])
data_test['intent'] = data_test['category'].cat.codes



# Extract the training and testing texts and labels
train_texts = data_train['query'].tolist()
train_labels = data_train['intent'].tolist()
test_texts = data_test['query'].tolist()
test_labels = data_test['intent'].tolist()

max_length = 128  # Adjust based on your dataset or model's max length

train_encodings = tokenizer.batch_encode_plus(
    train_texts, 
    truncation=True, 
    padding='max_length', 
    max_length=max_length, 
    return_tensors="tf",
    pad_to_max_length=True  # Explicitly enforce padding to max_length
)

test_encodings = tokenizer.batch_encode_plus(
    test_texts, 
    truncation=True, 
    padding='max_length', 
    max_length=max_length, 
    return_tensors="tf",
    pad_to_max_length=True  # Explicitly enforce padding to max_length
)



# Convert the labels to one-hot encoding
num_labels = len(data_train['category'].cat.categories)
train_labels_encoded = tf.one_hot(train_labels, num_labels)
test_labels_encoded = tf.one_hot(test_labels, num_labels)

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels_encoded))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), test_labels_encoded))

#Define the model architecture
input_ids = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='attention_mask')
output = model(input_ids, attention_mask=attention_mask)[0]
output = tf.keras.layers.Dense(num_labels, activation='softmax')(output[:, 0, :]) # Pooling the output
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

#Compile and train the model
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']


# Use smaller batch size
batch_size = 20



model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
model.fit(train_dataset.batch(batch_size), epochs=10)

# Evaluate the model

evaluation_results = model.evaluate(test_dataset.batch(batch_size))
accuracy = evaluation_results[1]  # Assuming accuracy is the second metric in the metrics list


Train for 690 steps
Epoch 1/10
690/690 [==============================] - 124s 180ms/step - loss: 1.2452 - accuracy: 0.9308
Epoch 2/10
690/690 [==============================] - 116s 169ms/step - loss: 1.1784 - accuracy: 0.9877
Epoch 3/10
690/690 [==============================] - 116s 169ms/step - loss: 1.1759 - accuracy: 0.9899
Epoch 4/10
690/690 [==============================] - 116s 168ms/step - loss: 1.1733 - accuracy: 0.9923
Epoch 5/10
690/690 [==============================] - 116s 168ms/step - loss: 1.1734 - accuracy: 0.9922
Epoch 6/10
690/690 [==============================] - 116s 168ms/step - loss: 1.1721 - accuracy: 0.9934
Epoch 7/10
690/690 [==============================] - 116s 168ms/step - loss: 1.1721 - accuracy: 0.9933
Epoch 8/10
690/690 [==============================] - 116s 168ms/step - loss: 1.1714 - accuracy: 0.9941
Epoch 9/10
690/690 [==============================] - 116s 168ms/step - loss: 1.1715 - accuracy: 0.9940
Epoch 10/10
35/35 [========================

In [38]:
# Insert data into the DataFrame
results_df = results_df.append({'model_name': 'DistilBert', 'validation_accuracy': accuracy}, ignore_index=True)

# Display the DataFrame
print(results_df)


            model_name  validation_accuracy
0                 LSTM             0.981429
1        Random Forest             0.974286
2  Logistic Regression             0.982857
3                  GRU             0.970000
4                  RNN             0.958571
5                 BERT             0.987143
6              Roberta             0.990000
7                XLnet             0.982857
8           DistilBert             0.990000


In [39]:
print(results_df)


            model_name  validation_accuracy
0                 LSTM             0.981429
1        Random Forest             0.974286
2  Logistic Regression             0.982857
3                  GRU             0.970000
4                  RNN             0.958571
5                 BERT             0.987143
6              Roberta             0.990000
7                XLnet             0.982857
8           DistilBert             0.990000


# Albert

In [41]:
from transformers import TFAlbertModel, AlbertTokenizer

tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
model = TFAlbertModel.from_pretrained('albert-base-v2')


# Load and preprocess the data
data_train = train_data_frame[['query', 'intent']]
data_train['category'] = pd.Categorical(data_train['intent'])
data_train['intent'] = data_train['category'].cat.codes

# Load and preprocess the data
data_test = validation_data_frame[['query', 'intent']]
data_test['category'] = pd.Categorical(data_test['intent'])
data_test['intent'] = data_test['category'].cat.codes



# Extract the training and testing texts and labels
train_texts = data_train['query'].tolist()
train_labels = data_train['intent'].tolist()
test_texts = data_test['query'].tolist()
test_labels = data_test['intent'].tolist()

max_length = 128  # Adjust based on your dataset or model's max length

train_encodings = tokenizer.batch_encode_plus(
    train_texts, 
    truncation=True, 
    padding='max_length', 
    max_length=max_length, 
    return_tensors="tf",
    pad_to_max_length=True  # Explicitly enforce padding to max_length
)

test_encodings = tokenizer.batch_encode_plus(
    test_texts, 
    truncation=True, 
    padding='max_length', 
    max_length=max_length, 
    return_tensors="tf",
    pad_to_max_length=True  # Explicitly enforce padding to max_length
)



# Convert the labels to one-hot encoding
num_labels = len(data_train['category'].cat.categories)
train_labels_encoded = tf.one_hot(train_labels, num_labels)
test_labels_encoded = tf.one_hot(test_labels, num_labels)

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels_encoded))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), test_labels_encoded))

#Define the model architecture
input_ids = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='attention_mask')
output = model(input_ids, attention_mask=attention_mask)[0]
output = tf.keras.layers.Dense(num_labels, activation='softmax')(output[:, 0, :]) # Pooling the output
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

#Compile and train the model
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']


# Use smaller batch size
batch_size = 20



model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
model.fit(train_dataset.batch(batch_size), epochs=10)

# Evaluate the model

evaluation_results = model.evaluate(test_dataset.batch(batch_size))
accuracy = evaluation_results[1]  # Assuming accuracy is the second metric in the metrics list


Train for 690 steps
Epoch 1/10
690/690 [==============================] - 231s 335ms/step - loss: 1.2434 - accuracy: 0.9273
Epoch 2/10
690/690 [==============================] - 218s 317ms/step - loss: 1.1827 - accuracy: 0.9833
Epoch 3/10
690/690 [==============================] - 218s 317ms/step - loss: 1.1794 - accuracy: 0.9867
Epoch 4/10
690/690 [==============================] - 218s 316ms/step - loss: 1.1785 - accuracy: 0.9869
Epoch 5/10
690/690 [==============================] - 218s 317ms/step - loss: 1.1780 - accuracy: 0.9880
Epoch 6/10
690/690 [==============================] - 218s 316ms/step - loss: 1.1788 - accuracy: 0.9869
Epoch 7/10
690/690 [==============================] - 218s 317ms/step - loss: 1.1771 - accuracy: 0.9884
Epoch 8/10
690/690 [==============================] - 218s 316ms/step - loss: 1.1746 - accuracy: 0.9909
Epoch 9/10
690/690 [==============================] - 218s 317ms/step - loss: 1.1747 - accuracy: 0.9906
Epoch 10/10
35/35 [========================

In [42]:
# Insert data into the DataFrame
results_df = results_df.append({'model_name': 'Albert', 'validation_accuracy': accuracy}, ignore_index=True)

# Display the DataFrame
print(results_df)


            model_name  validation_accuracy
0                 LSTM             0.981429
1        Random Forest             0.974286
2  Logistic Regression             0.982857
3                  GRU             0.970000
4                  RNN             0.958571
5                 BERT             0.987143
6              Roberta             0.990000
7                XLnet             0.982857
8           DistilBert             0.990000
9               Albert             0.995714


# Electra

In [43]:
from transformers import TFElectraModel, ElectraTokenizer

tokenizer = ElectraTokenizer.from_pretrained('google/electra-base-discriminator')
model = TFElectraModel.from_pretrained('google/electra-base-discriminator')



# Load and preprocess the data
data_train = train_data_frame[['query', 'intent']]
data_train['category'] = pd.Categorical(data_train['intent'])
data_train['intent'] = data_train['category'].cat.codes

# Load and preprocess the data
data_test = validation_data_frame[['query', 'intent']]
data_test['category'] = pd.Categorical(data_test['intent'])
data_test['intent'] = data_test['category'].cat.codes



# Extract the training and testing texts and labels
train_texts = data_train['query'].tolist()
train_labels = data_train['intent'].tolist()
test_texts = data_test['query'].tolist()
test_labels = data_test['intent'].tolist()

max_length = 128  # Adjust based on your dataset or model's max length

train_encodings = tokenizer.batch_encode_plus(
    train_texts, 
    truncation=True, 
    padding='max_length', 
    max_length=max_length, 
    return_tensors="tf",
    pad_to_max_length=True  # Explicitly enforce padding to max_length
)

test_encodings = tokenizer.batch_encode_plus(
    test_texts, 
    truncation=True, 
    padding='max_length', 
    max_length=max_length, 
    return_tensors="tf",
    pad_to_max_length=True  # Explicitly enforce padding to max_length
)



# Convert the labels to one-hot encoding
num_labels = len(data_train['category'].cat.categories)
train_labels_encoded = tf.one_hot(train_labels, num_labels)
test_labels_encoded = tf.one_hot(test_labels, num_labels)

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels_encoded))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), test_labels_encoded))

#Define the model architecture
input_ids = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='attention_mask')
output = model(input_ids, attention_mask=attention_mask)[0]
output = tf.keras.layers.Dense(num_labels, activation='softmax')(output[:, 0, :]) # Pooling the output
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

#Compile and train the model
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']


# Use smaller batch size
batch_size = 20



model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
model.fit(train_dataset.batch(batch_size), epochs=10)

# Evaluate the model

evaluation_results = model.evaluate(test_dataset.batch(batch_size))
accuracy = evaluation_results[1]  # Assuming accuracy is the second metric in the metrics list


Train for 690 steps
Epoch 1/10
690/690 [==============================] - 242s 351ms/step - loss: 1.2609 - accuracy: 0.9264
Epoch 2/10
690/690 [==============================] - 228s 330ms/step - loss: 1.1803 - accuracy: 0.9864
Epoch 3/10
690/690 [==============================] - 227s 329ms/step - loss: 1.1779 - accuracy: 0.9882
Epoch 4/10
690/690 [==============================] - 227s 329ms/step - loss: 1.1786 - accuracy: 0.9870
Epoch 5/10
690/690 [==============================] - 227s 329ms/step - loss: 1.1757 - accuracy: 0.9898
Epoch 6/10
690/690 [==============================] - 227s 329ms/step - loss: 1.1752 - accuracy: 0.9904
Epoch 7/10
690/690 [==============================] - 227s 330ms/step - loss: 1.1749 - accuracy: 0.9906
Epoch 8/10
690/690 [==============================] - 227s 329ms/step - loss: 1.1729 - accuracy: 0.9925
Epoch 9/10
690/690 [==============================] - 227s 329ms/step - loss: 1.1734 - accuracy: 0.9922
Epoch 10/10
35/35 [========================

In [44]:
# Insert data into the DataFrame
results_df = results_df.append({'model_name': 'Electra', 'validation_accuracy': accuracy}, ignore_index=True)

# Display the DataFrame
print(results_df)


             model_name  validation_accuracy
0                  LSTM             0.981429
1         Random Forest             0.974286
2   Logistic Regression             0.982857
3                   GRU             0.970000
4                   RNN             0.958571
5                  BERT             0.987143
6               Roberta             0.990000
7                 XLnet             0.982857
8            DistilBert             0.990000
9                Albert             0.995714
10              Electra             0.985714


# Final Evaluation

In [45]:
# Find the index of the row with the maximum validation accuracy
max_accuracy_index = results_df['validation_accuracy'].idxmax()

# Retrieve the model name with the maximum validation accuracy
best_model_name = results_df.loc[max_accuracy_index, 'model_name']

print(f"The best model with maximum validation accuracy is: {best_model_name}")

The best model with maximum validation accuracy is: Albert
